# MGIMO intensive

## OCR for text data

### 1. Libraries and parameters

Demo is based on [Tesseract OCR](https://github.com/tesseract-ocr/tesseract) framework.

In [ ]:
import os
import pytesseract
from tqdm.auto import tqdm
from pdf2image import convert_from_path
from PIL import Image

### 2. Document preprocessing

In [ ]:
def pdf2img(file_path, img_dir, first_page, last_page, dpi=200):
    """
    Turns pdf file to set of jpeg images.

    """
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
    pdf_pages = convert_from_path(
        pdf_path=file_path,
        dpi=dpi,
        output_folder=img_dir,
        first_page=first_page,
        last_page=last_page,
        fmt='JPEG'
    )
    return pdf_pages


def ocr_text(img_dir, lang='eng'):
    """
    Takes the text from image.

    """
    text = []
    for img_name in tqdm(sorted(os.listdir(img_dir))):
        if '.jpg' in img_name:
            text_tmp = str(
                pytesseract.image_to_string(
                    Image.open(f'{IMG_PATH}/{img_name}'),
                    lang=lang  # `eng+rus` for two languages in document
                )
            )
            text.append(text_tmp)
    return text

#### 2.1. Prepare for OCR

In [ ]:
IMG_PATH = 'scans'
PDF_PATH = '/home/jovyan/__DATA/mgimo_intensive/leo/Scan-140829-0001.pdf'

In [ ]:
NUM_PAGES = 10

pdf_pages = pdf2img(
    file_path=PDF_PATH,
    img_dir=IMG_PATH,
    first_page=1,
    last_page=NUM_PAGES
)

#### 2.2. Extract text with OCR

In [ ]:
text = ocr_text(img_dir=IMG_PATH, lang='rus+eng')

In [ ]:
for i, t in enumerate(text):
    print("=== Page", i, "===")
    print(t)

### 2. Text post-processing

#### 2.1. AI assistant init

In [ ]:
import json
from yandexgpt_client import YandexGPTClient

In [ ]:
def json_data(file_path):
    """Load and return JSON data from a file.
    
    Args:
        file_path (str): Path to the JSON file.
    
    Returns:
        dict or list: Parsed JSON data.
    """
    with open(file_path) as file:
        data = json.load(file)
    return data

In [ ]:
# Load credentials for YandexGPT access
access_data = json_data('/home/jovyan/__DATA/mgimo_intensive/.accessyagpt')

In [ ]:
instruction = """
Роль: Ты — опытный редактор и корректор, специализирующийся 
на научных текстах по биологии и экологии. Твоя задача — превратить
«сырой» результат распознавания сканированного документа в чистый, 
грамотный и структурированный текст, готовый к публикации или анализу.

Входные данные: Текст, полученный после OCR (оптического распознавания символов). 
Он содержит ошибки сканирования: лишние дефисы (переносы), случайные символы, 
слитное написание слов, неправильное распознавание букв, искажение таблиц 
и специальных символов.

Выходные данные: Исправленный, структурированный текст на русском языке, 
максимально соответствующий оригиналу.

Основные правила обработки

1. Исправление общих ошибок OCR
Склейка и разрывы: Если два слова написаны слитно («влесу»), раздели их. 
Если слово разорвано пробелом («гри бы»), объедини их в одно («грибы»), 
руководствуясь контекстом и словарем русского языка.
Переносы: Удали все технические переносы слов, расставленные для выравнивания строк. 
Если слово перенесено по слогам через дефис в конце строки (например, «дли-нным»), 
соедини части в целое слово («длинным»).
Цифры и буквы: Исправь типичную путаницу: буква «О» вместо цифры «0» (и наоборот), 
буква «З» вместо цифры «3», строчная «л» похожая на «П» и т.д.
Пример: «Было зарегистрировано 25 особей, температура воздуха 10°С» 
(а не «Было зарегистрировано 2 5 особей, температура воздуха 10°C»).

2. Работа с латинскими названиями (Специфика биоразнообразия)
Сохранение: Латинские названия видов (Rosa majalis), родов и семейств 
являются ключевыми и должны быть сохранены БЕЗ ПЕРЕВОДА.
Регистр и выделение: Приведи их к стандарту биологической номенклатуры: род с большой буквы, 
видовой эпитет — с маленькой (если это не имя собственное в устаревших названиях). 
Часто в текстах они выделяются курсивом. В твоем выводе можно использовать звездочки 
или подчеркивания для разметки, если формат это позволяет, или просто исправить раскладку и регистр.
Пример ошибки OCR: «B Rosa maj alis (шин овник)» -> Исправлено: «Rosa majalis (шиповник)».
Игнорирование переносов в латыни: Если латинское название разорвано переносом (Rosa maja- в 
конце строки и lis в начале следующей), соедини части без пробела и дефиса: Rosa majalis.

3. Таблицы
Распознавание: В сканах часто таблицы «съезжают». Постарайся восстановить структуру таблицы.
Форматирование: Если выходной формат поддерживает Markdown, оформи таблицу в Markdown-разметке. 
Если нет — используй выравнивание через табуляцию или пробелы, сохраняя колонки.
Содержимое: Проверь, чтобы данные в ячейках не «переползали» в соседние колонки из-за ошибок сканирования.
Пример искажения: «| Вид | Количество | | Rosa canina | 12 | Rosa | 5 |» 
-> Исправлено: «| Вид | Количество | | Rosa canina | 12 | Rosa majalis | 5 |».

4. Специальные символы
Температура: Символ градуса часто распознается как «°», «0», «о» или «C» (латинское). 
Исправляй на корректный символ градуса «°C» или «°С» в зависимости от контекста (русская «С» для шкалы Цельсия). 
Если в оригинале было «°C» (латиница), обычно оставляют как есть или меняют на общепринятый вид.
Пример: «t +10o C» -> «t +10 °C».
Единицы измерения: Проверь написание единиц (км, м, га, кг). Исправь «км» искаженное в «K.M» на «км».

5. Структура текста
Абзацы: Сохрани абзацное членение. Если в распознанном тексте абзацы пропали (весь текст сплошняком), 
попробуй восстановить логику (красная строка часто означает новый абзац).
Заголовки: Если видишь, что строка выделена жирным или написана заглавными буквами в оригинале 
(и это видно по смыслу), оформи её как заголовок (например, используя # в начале строки).

Алгоритм действий (Chain of Thought)
Анализ макета: Просмотри текст. Определи, где начинаются таблицы, где списки видов, 
а где обычный описательный текст.
Словарная чистка: Пройдись по тексту, исправляя очевидные опечатки и переносы. 
Используй контекст для разрешения неоднозначностей (например, «в лесу обитает лось» подскажет, 
что «влесу» — это два слова).
Обработка латыни: Найди все вкрапления латыни. Обычно они встречаются в скобках после 
русского названия или в списках видов. Восстанови их корректное написание.
Нормализация символов: Приведи спецсимволы к читаемому виду (°C, ±, × и т.д.).
Финальное вычитывание: Убедись, что текст выглядит как научная работа, а не как технический черновик.

Важные ограничения (Что НЕ нужно делать)
НЕ перефразируй: Твоя задача — исправить ошибки распознавания, а не переписать 
текст своими словами. Сохраняй авторский стиль и терминологию.
НЕ угадывай, если не уверен: Если слово или символ искажены настолько, что невозможно 
понять исходный вариант, оставь комментарий в квадратных скобках, например: [неразборчиво] 
или [искаженный символ]. Не заменяй это своим предположением без крайней необходимости.
НЕ трогай устаревшие названия: Если в тексте 80-х годов используется устаревшее название 
вида (синоним), оставь его как есть. Не заменяй на современный accepted name — это задача 
таксономической ревизии, а не постобработки.попросите пользователя предоставить больше информации.
"""

yaclient = YandexGPTClient(
    folder_id=access_data["folder_id"],
    api_key=access_data["api_key"],
    instruction_text=instruction
)

#### 2.2. Test one page processing

In [ ]:
# Ask our AI assistant

response = yaclient.call_yandexgpt(
    prompt=text[0], 
    model_name="yandexgpt", 
    max_tokens=4000, 
    temperature=0.1
)

In [ ]:
print(response)

#### 2.3. More pages for better quality

In [ ]:
big_text = '\n'.join(text[0:3])

response = yaclient.call_yandexgpt(
    prompt=big_text, 
    model_name="yandexgpt", 
    max_tokens=4000, 
    temperature=0.1
)

In [ ]:
print(response)

#### 2.4. Combine to pipeline

In [ ]:
STEP = 3
final_text = []

for i in range(len(text) // STEP + 1):
    # Range for pages
    start_page = i * STEP
    end_page = min((i + 1) * STEP, len(text))
    print("Pages from", start_page, "to", end_page, end="")

    # Process by several pages
    big_text = '\n'.join(text[start_page : end_page])
    response = yaclient.call_yandexgpt(
        prompt=big_text, 
        model_name="yandexgpt", 
        max_tokens=4000, 
        temperature=0.1
    )
    final_text.append(response)
    print(" --> done")

In [ ]:
print('\n'.join(final_text))

In [ ]:
# Write result to file

with open('text.txt', 'w', encoding='utf-8') as file:
    for line in final_text:
        file.write(line + '\n') 